In [ ]:
# Covid19 Sound

In [3]:
import pandas as pd
df = pd.read_pickle("F:/DATAS/covid-19-main/dataset-main/covid19_split_balancevalid_df.pkl")
df.shape

(2733, 3)

In [ ]:
print(df.groupby("label")["melspectrogram"].count())
for i in range(11):
    df_valid = df.where(df["fold"]==i)
    print(df_valid.groupby("label")["melspectrogram"].count())

In [ ]:
# Coughvid Sound

In [1]:
import pandas as pd
df = pd.read_pickle("F:/DATAS/COUGHVID-public_dataset_v3/coughvid_fine_df.pkl")
print(df.shape)
print(df)
print(df.groupby("label")["melspectrogram"].count())

(2850, 3)
                                         melspectrogram  label  fold
0     [[-80.0, -80.0, -80.0, -80.0, -80.0, -80.0, -8...      0     3
1     [[-80.0, -80.0, -80.0, -80.0, -80.0, -80.0, -8...      0     4
2     [[-80.0, -80.0, -80.0, -80.0, -80.0, -80.0, -8...      0     6
3     [[-80.0, -80.0, -80.0, -80.0, -80.0, -80.0, -8...      0     2
4     [[-80.0, -80.0, -80.0, -80.0, -80.0, -80.0, -8...      0     5
...                                                 ...    ...   ...
2845  [[-80.0, -80.0, -80.0, -80.0, -80.0, -80.0, -8...      0     6
2846  [[-80.0, -80.0, -80.0, -80.0, -80.0, -80.0, -8...      0     6
2847  [[-80.0, -80.0, -80.0, -80.0, -80.0, -80.0, -8...      0     7
2848  [[-80.0, -80.0, -80.0, -80.0, -80.0, -80.0, -8...      0     2
2849  [[-80.0, -80.0, -80.0, -80.0, -80.0, -80.0, -8...      0     1

[2850 rows x 3 columns]
label
0    2076
1     774
Name: melspectrogram, dtype: int64


In [4]:
for i, row in enumerate(df.iterrows()):
    # df_valid = df.where(df["fold"]==i)
    # print(df_valid.groupby("label")["melspectrogram"].count())
    print(row[1][0].shape)
    if i > 2:
        break
        

(128, 128)
(128, 128)
(128, 128)
(128, 128)


In [ ]:
import os
import random
import librosa
import numpy as np
import pandas as pd

from tqdm import tqdm

In [ ]:
HOP_LENGTH = 512        # number of samples between successive frames
WINDOW_LENGTH = 512     # length of the window in samples
N_MEL = 128             # number of Mel bands to generate


def compute_melspectrogram_with_fixed_length(audio, sampling_rate, num_of_samples=128):
    try:
        # compute a mel-scaled spectrogram
        melspectrogram = librosa.feature.melspectrogram(y=audio, 
                                                        sr=sampling_rate, 
                                                        hop_length=HOP_LENGTH,
                                                        win_length=WINDOW_LENGTH, 
                                                        n_mels=N_MEL)

        # convert a power spectrogram to decibel units (log-mel spectrogram)
        melspectrogram_db = librosa.power_to_db(melspectrogram, ref=np.max)
        
        melspectrogram_length = melspectrogram_db.shape[1]
        
        # pad or fix the length of spectrogram 
        if melspectrogram_length != num_of_samples:
            melspectrogram_db = librosa.util.fix_length(melspectrogram_db, 
                                                        size=num_of_samples, 
                                                        axis=1, 
                                                        constant_values=(0, -80.0))
    except Exception as e:
        print("\nError encountered while parsing files\n>>", e)
        return None 
    
    return melspectrogram_db

# DCASE2024

In [ ]:
root_path = "F:/DATAS/DCASE2024Task2ASD/"
mts = ["bearing", "fan", "gearbox", "slider", "ToyCar", "ToyTrain", "valve"]
l2m = {0:"bearing", 1:"fan", 2:"gearbox", 3:"slider", 4:"ToyCar", 5:"ToyTrain", 6:"valve"}
m2l = {"bearing":0, "fan":1, "gearbox":2, "slider":3, "ToyCar":4, "ToyTrain":5, "valve":6}
subpath = [root_path+f"dev_{mt}/{mt}/train/" for mt in mts]
with open("./dcase2024cls.csv", 'w') as fout:
    fout.write(",slice_file_name,fold,classID\n")
    idx = 0
    for j,filepath in enumerate(subpath):
        for item in os.listdir(filepath):
            fout.write(f"{idx},{item},{np.random.randint(1, 11)},{j}\n")
            idx+=1
print("end")

In [ ]:
# load the csv metadata file into a Pandas DataFrame structure
DCASE2024_METADATA_PATH = "./dcase2024cls.csv"
dcase2024_metadata_df = pd.read_csv(DCASE2024_METADATA_PATH,
                               usecols=["slice_file_name", "fold", "classID"],
                               dtype={"fold": "str", "classID" : "uint8"})

dcase2024_metadata_df

# COUGHVID 切分为多段然后截断到
- duration：1.5s
- length：32306
- sample rate：22050

In [ ]:
metadata_path = "C:/Program Files (zk)/PythonFiles/AClassification/SoundDL-CoughVID/datasets/waveinfo_fewtoml_split.csv"
metadata = pd.read_csv(metadata_path, delimiter=',', header=0, index_col=0)
metadata

In [ ]:
metadata = metadata.iloc[:, [0, 1]]
m2l = {"healthy":0, "COVID-19":1}
metadata["status"] = metadata["status"].apply(lambda x: m2l[x])
print(metadata.head(10))
# datadf1.where(datadf1["cough_detected"]>0.65).where(datadf1["duration"] < 13).where(datadf1["duration"] > 0.86).dropna(axis=0)
print(metadata.groupby("status")["uuid"].count())
# metadata

In [ ]:
root_path = "F:/DATAS/COUGHVID-public_dataset_v3/coughvid_20211012_fine/"
y, sr = librosa.load(root_path+"sound0013_727349f2-4730-45f6-99fe-8cf1ae1f2004.wav")
print(y.shape, sr)
print(len(y)/sr)

In [ ]:
foldcol = []
N = len(metadata)
fold_per = N // 10
for i in range(1, 10):
    foldcol.extend([i]*fold_per)
foldcol.extend([10]*(N-len(foldcol)))

random.shuffle(foldcol)
metadata["fold"] = foldcol
metadata.reset_index(drop=True)
metadata

In [ ]:
metadata.groupby("status")["uuid"].count()

In [ ]:
# SOUND_DURATION = 2.95   # fixed duration of an audio excerpt in seconds
features = []
sample_rate = 22050
# iterate through all dataset examples and compute log-mel spectrograms
for index, row in tqdm(metadata.iterrows(), total=len(metadata)):
    file_path = root_path + row["uuid"] + ".wav"
    sample_rate = 22050
    
    audioseg, _ = librosa.load(file_path)
    
    # audio, sample_rate = librosa.load(file_path, duration=SOUND_DURATION, res_type='kaiser_fast')
    melspectrogram = compute_melspectrogram_with_fixed_length(audioseg, sample_rate)
    print(melspectrogram.shape)
    label = row["status"]
    fold = row["fold"]
    
    features.append([melspectrogram, label, fold])


In [ ]:
# convert into a Pandas DataFrame 
us8k_df = pd.DataFrame(features, columns=["melspectrogram", "label", "fold"])
us8k_df.to_pickle(root_path+"coughvid_fine_df.pkl")

In [ ]:
import pandas as pd
data_df = pd.read_pickle("F:/DATAS/COUGHVID-public_dataset_v3/coughvid_fine_df.pkl")
data_df

In [ ]:
data_df.groupby("label").count()

In [ ]:
for index, row in tqdm(data_df.iterrows(), total=len(data_df)):
    print(row["melspectrogram"].shape)
    break

# COUGHVID 去除静音然后截断到48000长度

duration：2.95s
sample rate：16000

In [ ]:
metadata_path = "C:/Program Files (zk)/PythonFiles/AClassification/SoundDL-CoughVID/datasets/waveinfo.csv"
metadata = pd.read_csv(metadata_path, delimiter=',', header=0, index_col=0)
metadata = metadata.where(metadata["cough_detected"]>0.65).where(metadata["duration"] < 13).where(metadata["duration"] > 0.86).dropna(axis=0)
# metadata = metadata.iloc[:, [0, 2, 9, 10]]

# datadf1.where(datadf1["cough_detected"]>0.65).where(datadf1["duration"] < 13).where(datadf1["duration"] > 0.86).dropna(axis=0)
print(metadata.groupby("status")["filename"].count())
# metadata

In [ ]:
root_path = "C:/Program Files (zk)/PythonFiles/AClassification/SoundDL-CoughVID/datasets/"
df = pd.read_csv(root_path+"waveinfo_labedfine_forcls.csv", delimiter=',', header=0, index_col=0)
print(df.groupby("status_full")["filename"].count())
cough_metadata_df = df.iloc[:, [0, 6]]
# df = pd.read_csv(root_path+"waveinfo_labedfine_staaSSL.csv", delimiter=',', header=0, index_col=0)
# df.groupby("status_full")["filename"].count()
# iterate through all dataset examples and compute log-mel spectrograms
foldcol = []
N = len(cough_metadata_df)
fold_per = N // 10
for i in range(1, 10):
    foldcol.extend([i]*fold_per)
foldcol.extend([10]*(N-len(foldcol)))

random.shuffle(foldcol)
cough_metadata_df["fold"] = foldcol
cough_metadata_df.reset_index(drop=True)
cough_metadata_df = cough_metadata_df.iloc[:, [0, 2, 1]]
cough_metadata_df.rename(columns={"status_full": "classID"}, inplace=True)
cough_metadata_df

In [ ]:
import sys
sys.path.append(r'C:/Program Files (zk)/PythonFiles/AClassification/AudioClassification-Pytorch-KZhao/')
from ackit.data_utils.audio import AudioSegment

In [ ]:
SOUND_DURATION = 2.95   # fixed duration of an audio excerpt in seconds
features = []
root_path = "F:/DATAS/COUGHVID-public_dataset_v3/"


# iterate through all dataset examples and compute log-mel spectrograms
for index, row in tqdm(cough_metadata_df.iterrows(), total=len(cough_metadata_df)):
    file_path = row["filename"]
    sample_rate = 22050
    
    audioseg = AudioSegment.from_file(file_path)
    audioseg.vad()
    audioseg.resample(target_sample_rate=sample_rate)
    audioseg.crop(duration=2.95, mode="eval")
    
    # audio, sample_rate = librosa.load(file_path, duration=SOUND_DURATION, res_type='kaiser_fast')
    melspectrogram = compute_melspectrogram_with_fixed_length(audioseg.samples, sample_rate)
    label = row["classID"]
    fold = row["fold"]
    
    features.append([melspectrogram, label, fold])

# convert into a Pandas DataFrame 
us8k_df = pd.DataFrame(features, columns=["melspectrogram", "label", "fold"])
us8k_df.to_pickle(root_path+"coughvid_df.pkl")

In [ ]:
import pandas as pd
data_df = pd.read_pickle("F:/DATAS/COUGHVID-public_dataset_v3/coughvid_df.pkl")

In [ ]:
data_df

In [ ]:
for i in range(1, 11):
    print(data_df[data_df["fold"]==i].groupby("label")["fold"].count())

# DCASE2020

In [ ]:
SOUND_DURATION = 2.95   # fixed duration of an audio excerpt in seconds

features = []

root_path = "F:/DATAS/DCASE2024Task2ASD/"


# iterate through all dataset examples and compute log-mel spectrograms
for index, row in tqdm(dcase2024_metadata_df.iterrows(), total=len(dcase2024_metadata_df)):
    subpath = f"/dev_{l2m[row['classID']]}/{l2m[row['classID']]}/train"
    file_path = f"{root_path}"+subpath+"/{row["slice_file_name"]}"
    audio, sample_rate = librosa.load(file_path, duration=SOUND_DURATION, res_type='kaiser_fast')
    
    melspectrogram = compute_melspectrogram_with_fixed_length(audio, sample_rate)
    label = row["classID"]
    fold = row["fold"]
    
    features.append([melspectrogram, label, fold])

# convert into a Pandas DataFrame 
us8k_df = pd.DataFrame(features, columns=["melspectrogram", "label", "fold"])
us8k_df.to_pickle(root_path+"us8k_df.pkl")